# Inventories

Inventories are a fundamental doc entrypoint for our infrastructures. 

They contain a lot of informations, including:
    
    - ansible_user
    - configuration variables in [group_name:vars]
    - host grouping eg. by geographical zones in [group_name:children]
    
    
Files:


  - [inventory](/edit/notebooks/exercise-05/inventory)
    

In [1]:
cd /notebooks/exercise-05

/notebooks/exercise-05


In [2]:
!cat inventory

# I can group hosts in inventory

[web:children]
web_rome
web_milan

[web_rome:children]
web_rome_test
web_rome_prod

[web_rome_prod]
172.23.0.[3:4]

[web_milan]
172.24.0.[5:6]

# further host variables
[web_rome:vars]
ansible_username=root 


# Connect with docker 
[web_rome_test:vars]
ansible_connection=docker
ansible_docker_extra_args="-Htcp://172.17.0.1"

# The actual host reference
[web_rome_test]
ansible101_web_1

# 
# Don't need to be Ssh 
#  to be my local machine ;)
#
[course]
localhost    ansible_connection=local 

The `ansible` executable can process inventory files

In [3]:
!ansible -i inventory --list-host all

  hosts (6):
    172.23.0.3
    172.23.0.4
    172.24.0.5
    172.24.0.6
    ansible101_web_1
    localhost


#### Exercise

Use `ansible` to show:
    
    - all hosts of the web group.

In [11]:
# Use this  cell for the exercise

In [ ]:
# The ping module is very useful. 
# Use it whenever you want to check connectivity!
!ansible -m ping -i inventory web_ome

## Inventory scripts

In [ ]:
#To create custom inventory scripts just use python ;) and set it in
!grep inventory  ansible.cfg # inventory = ./docker-inventory.py

#### Exercise

in the official ansible documentation find at least 3 `ansible_connection=docker` parameters 

In [13]:
"""List our containers. 

  Note: this only works with docker-compose containers.

"""
from __future__ import print_function
# 
# Manage different docker libraries
#
try:
    from docker import Client
except ImportError:
    from docker import APIClient as Client


c = Client(base_url="http://172.17.0.1:2375")

# Define a function to make it clear!
container_fmt = lambda x: (
    x['Names'][0][1:],
    x['Labels']['com.docker.compose.service'], 
    x['NetworkSettings']['Networks']['bridge']['IPAddress'],
)

for x in c.containers():
    try:
        print(*container_fmt(x), sep='\t\t')
    except KeyError:
        # skip non-docker-compose containers
        pass

ansible101_dev_1		dev		172.17.0.2
ansible101_web_2		web		172.17.0.3
ansible101_web_1		web		172.17.0.4
ansible101_bastion_1		bastion		172.17.0.5


In [ ]:
# Ansible accepts
import json

inventories = {
    'web': {
        'hosts': ['ws-1', 'ws-2'],
    },
    'db': {
        'hosts': ['db-1', 'db-2'],
    }
}

# like this 
print(json.dumps(inventories, indent=1))
      

In [ ]:
# You can pass variables to generated inventories too
inventories['web']['host_vars'] = {
    'ansible_ssh_common_args': ' -o GSSApiAuthentication=no'
}
print(json.dumps(inventories, indent=1))

#### Exercise: 

Reuse the code in [inventory-docker.py](/edit/notebooks/exercise-05/inventory-docker.py) to print a json inventory that:

  - connects via docker to "web" hosts
  - connects via ssh to "ansible" hosts 

Test it in the cell below.

** NOTE: there's a [docker inventory](https://github.com/ansible/ansible/blob/devel/contrib/inventory/docker.py) script shipped with ansible **

In [ ]:
!ansible -m ping -i inventory-docker-solution.py all 

#### Exercise

Modify the [inventory-docker.py](/edit/notebooks/exercise-05/inventory-docker.py) to skip StrictHostKeyChecking only on web hosts.

In [ ]:
# Test here your inventory

# Configurations

You may want to split inventory files and separate prod and test environment.

#### Exercise: 

split inventory in two inventory files:

  - [prod](/edit/notebooks/exercise-05/prod)  for production servers 
  - [test](/edit/notebooks/exercise-05/test)  for test servers
  
Then use `ansible -i` to explicitly use the different ones.

In [ ]:
# Use this cell to test the exercise

## group_vars and host_vars

You can move variables out of inventories - eg to simplify inventory scripts - and store them in files:

 - under `group_vars` for host groups
 - under `host_vars` for single hosts

In [ ]:
!tree group_vars

If you have different inventories, you can store different set of variable in custom files.
The `all` ones will be shared between all inventories

#### Exercise: 
    
  - edit [group_vars/all](/edit/notebooks/exercise-05/group_vars/all) and move there all common variables from [inventory](/edit/notebooks/exercise-05/inventory) 

In [ ]:
# Test here the new inventory file

Inventory variables can store almost everything and even describe the architecture of your deployment


In [ ]:
!cat group_vars/example

We can even mix and mojo `group_vars` and `inventory`, as we'll see in the next lessons.





### host_vars

Host vars can be used in automated or cloud deployments where:

  - every new host or vm, at boot, populate its own entries in `host_vars` (Eg. via file)
  - ansible is run after that setup and uses `host_vars` to configure the server and expose that values to the other machines.